In [1]:
# begin importing of standard modules
import re
import random
import os
import tkinter as tk
from tkinter import messagebox
import pandas as pd
import pprint as pp
import sys
# end importing of standard modules

# begin setting of home path variable and adding data directory to path import list
if 'Step_1.ipynb' or 'Step_1.py' or 'Step_1.exe' in os.listdir():
    home = os.getcwd()     
sys.path.append(os.path.join(home, 'data')) #This will add the data folder to the path variable so modules can be imported 
# end setting of home path variable and adding data directory to path import list

# begin creation Regular Expressions
validHash = re.compile(r'#\d{0,3}')

pinRegex1 = re.compile(r'GPIO_PIN_(\d{0,2})')
pinRegex2 = re.compile(r'GPIO_InitStruct.Pin')


modeRegex = re.compile(r'''
            GPIO_MODE_                  #This will match the first part
            (
            INPUT|
            OUTPUT_PP|
            OUTPUT_OD|
            AF_PP|
            AF_OD|
            ANALOG|
            IT_RISING|
            IT_FALLING|
            IT_RISING_FALLING|
            EVT_RISING|
            EVT_FALLING|
            EVT_RISING_FALLING)    
            ''', re.VERBOSE)

pullRegex = re.compile(r'''
            GPIO_NOPULL|
            GPIO_PULLUP|
            GPIO_PULLDOWN
            ''', re.VERBOSE)

speedRegex = re.compile(r'''
             GPIO_SPEED_FREQ_LOW|
             GPIO_SPEED_FREQ_MEDIUM|
             GPIO_SPEED_FREQ_HIGH|
             GPIO_SPEED_FREQ_VERY_HIGH
             ''', re.VERBOSE)

gpioInitRegex = re.compile(r'''
                 HAL_GPIO_Init.*GPIO([A-K])
                 ''', re.VERBOSE)
# end creation Regular Expressions

# begin import of created modules
try:   
    import ds
    import mypath
except ModuleNotFoundError:
   pass
# begin import of created modules

# begin importing and creation of a data frame
os.chdir(mypath.data)
df = pd.read_csv('data.csv')
# end importing and creation of a data frame

pd.set_option('max_colwidth', 200)
df

,std,hsh,Perfect Repository Upload,CorePath
0,20416318,#6,False,Invalid Repository
1,21957029,#5,False,Invalid Repository
2,22046364,#4,True,C:\Users\1995m\Desktop\skripsieCode\stdRepos\#4\Core
3,22074376,#1,False,C:\Users\1995m\Desktop\skripsieCode\stdRepos\#1\Da-Bug\Core
4,22127011,#2,False,C:\Users\1995m\Desktop\skripsieCode\stdRepos\#2\First_Project\Core
5,22218521,#3,True,C:\Users\1995m\Desktop\skripsieCode\stdRepos\#3\Core
6,22252975,#0,True,C:\Users\1995m\Desktop\skripsieCode\stdRepos\#0\Core


In [2]:
# begin adding a path to main.c column to dataframe
df['main.c Path'] = 'No main.c file. Double check student repository for correct upload!'
# end adding a path to main.c column to dataframe

# begin function that identifies the pinout in main.c
def goIntoMain(pathToC, i):
    try:
        cfile = open(pathToC)
    except FileNotFoundError:
        df.loc[df['hsh'] == i, 'main.c Path'] = 'No main.c file. Double check student repository for correct upload!'
        pass
# end function that identifies the pinout in main.c

# begin looping through each student repo to find the main.c file and pass it to the function that goes into main and gets the pinouts
os.chdir(mypath.repos)
for i in os.listdir():
#     import ipdb; ipdb.set_trace()
    pathToCore = df.loc[df['hsh'] == i, 'CorePath'].values[0]
    if os.path.isdir(pathToCore):
        pathToCFile = os.path.join(pathToCore, 'Src', 'main.c')
        df.loc[df['hsh'] == i, 'main.c Path'] = pathToCFile
        goIntoMain(pathToCFile , i)
    else:
        pass
os.chdir(mypath.repos)
# end looping through each student repo to find the main.c file and pass it to the function that goes into main and gets the pinouts

In [3]:
testGoMain = df.loc[df['hsh'] == '#3', 'main.c Path'].values[0]
cFileT = open(testGoMain)
lines = cFileT.readlines()
lines

['/* USER CODE BEGIN Header */\n',
 '/**\n',
 '  ******************************************************************************\n',
 '  * @file           : main.c\n',
 '  * @brief          : Main program body\n',
 '  ******************************************************************************\n',
 '  * @attention\n',
 '  *\n',
 '  * <h2><center>&copy; Copyright (c) 2020 STMicroelectronics.\n',
 '  * All rights reserved.</center></h2>\n',
 '  *\n',
 '  * This software component is licensed by ST under BSD 3-Clause license,\n',
 '  * the "License"; You may not use this file except in compliance with the\n',
 '  * License. You may obtain a copy of the License at:\n',
 '  *                        opensource.org/licenses/BSD-3-Clause\n',
 '  *\n',
 '  ******************************************************************************\n',
 '  */\n',
 '/* USER CODE END Header */\n',
 '\n',
 '/* Includes ------------------------------------------------------------------*/\n',
 '#include "main.h"\

In [4]:
# testFile = open(os.path.join(mypath.data, 'test.c'))
# lines = testFile.readlines()
# lines

In [5]:
dfPins = pd.DataFrame(columns = ['Pins Used', 'Mode','Pull-Type','Speed']) 


pinList = []
validList = []
modeList = []
pullList = []
speedList = []
x = 0

for i in lines:
#     import ipdb; ipdb.set_trace()
    isValid = pinRegex2.findall(i)
    isPinLine = pinRegex1.findall(i)
    isModeLine = modeRegex.findall(i)
    isPullLine = pullRegex.findall(i)
    isSpeedLine = speedRegex.findall(i)
    isGpioInitLine = gpioInitRegex.findall(i)
    if isPinLine and isValid:
        pinList.extend(isPinLine)
        validList.extend(isValid)      
    elif isModeLine:
        modeList.extend(isModeLine)
    elif isPullLine:
        pullList.extend(isPullLine)
    elif isSpeedLine:
        speedList.extend(isSpeedLine)
    elif isGpioInitLine and pinList and validList:
        dfTempPin = pd.DataFrame(columns = ['Pins Used', 'Mode','Pull-Type','Speed']) 
#         print(pinList)
#         print(modeList)
#         print(pullList)
#         print(speedList)
        for i in pinList:
            dfPins.loc[x] = ['P' + isGpioInitLine[0] + i] + [modeList] + [pullList] + [speedList]
            x += 1           
        pinList = []
        modeList = []
        pullList = []
        speedList = []
        validList = []
    else:
        pass


In [10]:
dfPins

,Pins Used,Mode,Pull-Type,Speed
0,PC13,[IT_RISING],[GPIO_NOPULL],[]
1,PC3,[OUTPUT_PP],[GPIO_NOPULL],[GPIO_SPEED_FREQ_VERY_HIGH]
2,PA5,[OUTPUT_PP],[GPIO_NOPULL],[GPIO_SPEED_FREQ_LOW]
3,PA9,[OUTPUT_PP],[GPIO_NOPULL],[GPIO_SPEED_FREQ_LOW]
4,PA6,[IT_RISING],[GPIO_PULLDOWN],[]
5,PA7,[IT_RISING],[GPIO_PULLDOWN],[]
6,PA8,[IT_RISING],[GPIO_PULLDOWN],[]
7,PA10,[IT_RISING],[GPIO_PULLDOWN],[]
8,PC7,[OUTPUT_PP],[GPIO_NOPULL],[GPIO_SPEED_FREQ_LOW]
9,PB4,[OUTPUT_PP],[GPIO_NOPULL],[GPIO_SPEED_FREQ_LOW]


In [7]:
string = (r'   GPIO_InitStruct.Pin = GPIO_PIN_6|GPIO_PIN_7|GPIO_PIN_8|GPIO_PIN_10')

In [8]:
testGPIO = re.compile(r'GPIO_PIN_(\d{0,2})')
testasd = re.compile(r'GPIO_InitStruct.Pin')

In [9]:
pinRegex1.findall(string)
pinRegex2.findall(string)

['GPIO_InitStruct.Pin']